### импорт модулей


In [1]:
import re
import nltk
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from collections import Counter 
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.utils import shuffle
import string
lst_stopwords = nltk.corpus.stopwords.words('russian')
lst_stopwords.extend(['…', '«', '»', '...'])


In [21]:
import session_info
session_info.show()

### импорт файла (поменяйте на / все \  и оставьте r) 


In [2]:
def import_file(file_path):
    df_raw = pd.read_csv(file_path)
    df_raw.dropna(inplace=True)
    df_raw.rename(columns={'text': 'content'}, inplace=True)
    return df_raw

In [3]:
df_raw=import_file(r"C:/Users/Jora/Desktop/документы/jupyter_notes/lenta-ru-news.csv")[:10000]


<ipython-input-3-06df1d87a052>:1: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  df_raw=import_file(r"C:/Users/Jora/Desktop/документы/jupyter_notes/lenta-ru-news.csv")[:10000]


In [4]:
df_raw #весь датасет

,url,title,content,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18
...,...,...,...,...,...,...
9995,https://lenta.ru/news/2000/07/10/kpp/,На границе Дагестана и Чечни будут стрелять в ...,Глава МВД Дагестана приказал нарядам милиции н...,Россия,Все,2000/07/10
9996,https://lenta.ru/news/2000/07/10/gossovet/,Путин пересадит губернаторов в Госсовет,Представители Кремля ведут активный диалог с г...,Россия,Все,2000/07/10
9997,https://lenta.ru/news/2000/07/10/clowns/,В Москву съедутся клоуны со всего мира,"В Москве 20 июля открывается фестиваль ""Планет...",Россия,Все,2000/07/10
9998,https://lenta.ru/news/2000/07/10/women/,Старшая Уильямс выигрывает чемпионский титул,Чисто американским получился финал Уимблдонск...,Спорт,Все,2000/07/10


In [5]:
def clean_text(text, tokenizer, stopwords):

    text = str(text).lower()  
    text = re.sub(r"\[(.*?)\]", "", text)  
    text = re.sub(r"\s+", " ", text)  
    text = re.sub(r"\w+…|…", "", text)  
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  

    tokens = tokenizer(text)  
    tokens = [t for t in tokens if not t in lst_stopwords]  
    tokens = ["" if t.isdigit() else t for t in tokens]  
    tokens = [t for t in tokens if len(t) > 1] 
    return tokens

In [6]:
def prep_tokens(df_raw):
    text_columns = ["title", "tags", "content"]
    df_raw['content'] = df_raw['content'].fillna(" ")
    for col in text_columns:
        df_raw[col] = df_raw[col].astype(str)
    # создаем текст основанный на content title и tag
    df_raw["text"] = df_raw[text_columns].apply(lambda x: " | ".join(x), axis=1)
    df_raw["tokens"] = df_raw["text"].map(lambda x: clean_text(x, word_tokenize, lst_stopwords))
    _, idx = np.unique(df_raw["tokens"], return_index=True)
    df_raw = df_raw.iloc[idx, :]

    # Remove empty values
    df_raw = df_raw.loc[df_raw.tokens.map(lambda x: len(x) > 0), ["text", "tokens"]]
    return df_raw
    

In [7]:
prep_data=prep_tokens(df_raw)

In [8]:
docs = prep_data["text"].values
tokenized_docs = prep_data["tokens"].values
vocab = Counter()
for token in tokenized_docs:
    vocab.update(token)

In [9]:
def vectorize(list_of_docs, model):
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features

### создаем модель и получаем вектора для кластерного анализа

In [10]:
model = Word2Vec(sentences=prep_data['tokens'].values, size=100, window=5, min_count=1, workers=4)

In [11]:
vectorized_docs= vectorize(prep_data['tokens'], model=model)
len(vectorized_docs), len(vectorized_docs[0])

(9999, 100)

In [12]:
vectorized_docs

[array([ 0.43925145, -0.57568204, -0.5212376 , -0.4245282 , -0.14769061,
         0.6126115 , -0.03091953, -0.75611305,  0.18251371, -0.28290483,
         0.33669513, -0.26896107, -0.40011087, -0.3989204 ,  0.9133078 ,
         0.40275693,  0.36256078,  0.22565603,  0.16840705, -0.6481296 ,
        -0.71697456,  0.42276287,  0.07113071, -0.80553865,  0.05312611,
        -0.59827083, -0.1510405 ,  0.1643429 , -0.07270619,  0.6775989 ,
         0.1907997 , -0.20175415, -0.17012255,  0.21772045,  0.6527795 ,
        -0.15380022, -0.63424075,  0.73850846,  0.8801497 , -0.18646023,
         1.121822  ,  0.04802516,  0.7233884 ,  0.22571443, -0.0199764 ,
        -0.7761431 , -0.7555809 , -0.03859783, -0.13021311,  0.18157633,
        -0.34106073,  0.67772615,  0.12505035, -0.13453981, -0.49026898,
         0.2185742 , -0.23703472,  0.34543818,  0.76264316, -0.1249436 ,
         0.32767403,  0.01141791, -0.34950915,  0.37856975,  0.5117017 ,
         0.17030227,  0.10415482,  0.16297622,  0.4

In [13]:
prep_data['vectors'] =vectorized_docs
prep_data.fillna('')

,text,tokens,vectors
3514,3а 1999 год ФСБ поймала 42 хакера | Все | В Фе...,"[3а, год, фсб, поймала, хакера, федеральной, с...","[0.43925145, -0.57568204, -0.5212376, -0.42452..."
7099,Adidas и Nike предъявили иск торговому концерн...,"[adidas, nike, предъявили, иск, торговому, кон...","[0.1791423, -0.35473675, -0.44891372, -0.31057..."
9608,AFP опровергает данные о потерях под Сержень-Ю...,"[afp, опровергает, данные, потерях, сержень, ю...","[0.7999505, -0.49795753, -0.7573627, -0.830441..."
2876,Agence France Presse пытается найти себя в Инт...,"[agence, france, presse, пытается, найти, инте...","[0.34177557, -0.33457664, -0.6499758, -0.43071..."
7295,Air France отменит авиарейсы из-за забастовки ...,"[air, france, отменит, авиарейсы, забастовки, ...","[0.13892676, -0.40690082, -0.59749925, -0.3341..."
...,...,...,...
6831,Ястржембский: Трошев назначен командующим ОГВ ...,"[ястржембский, трошев, назначен, командующим, ...","[1.0650803, -0.6105986, -0.68620586, -0.918035..."
1094,Ястржембский учит Ельцина уважать руководство ...,"[ястржембский, учит, ельцина, уважать, руковод...","[0.69636613, -0.42983907, -0.6146447, -0.57216..."
9604,"Яшин должен вернуть ""Оттаве"" год | Все | Извес...","[яшин, должен, вернуть, оттаве, год, известный...","[0.1666738, -0.33385435, -0.58110315, -0.34407..."
6477,Яшин не поможет сборной России | Все | Как это...,"[яшин, поможет, сборной, россии, это, печально...","[0.2968484, -0.33532962, -0.5001712, -0.260335..."


In [14]:
def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):

    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [15]:
clustering, cluster_labels = mbkmeans_clusters(X=prep_data['vectors'].tolist(),k=22, print_silhouette_values=True)
df_clusters = pd.DataFrame({
    "text": docs,
    "tokens": [" ".join(text) for text in tokenized_docs],
    "cluster": cluster_labels
})

C:\Users\Jora\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 3072 or by setting the environment variable OMP_NUM_THREADS=2
  warnings.warn(


For n_clusters = 22
Silhouette coefficient: 0.13
Inertia:5349.153186828742
Silhouette values:
    Cluster 12: Size:1039 | Avg:0.23 | Min:-0.01 | Max: 0.43
    Cluster 17: Size:346 | Avg:0.20 | Min:-0.02 | Max: 0.42
    Cluster 21: Size:336 | Avg:0.18 | Min:-0.09 | Max: 0.43
    Cluster 16: Size:111 | Avg:0.15 | Min:-0.07 | Max: 0.38
    Cluster 7: Size:793 | Avg:0.15 | Min:-0.09 | Max: 0.41
    Cluster 0: Size:199 | Avg:0.14 | Min:-0.13 | Max: 0.39
    Cluster 8: Size:214 | Avg:0.14 | Min:-0.10 | Max: 0.35
    Cluster 10: Size:179 | Avg:0.13 | Min:-0.14 | Max: 0.36
    Cluster 18: Size:422 | Avg:0.13 | Min:-0.12 | Max: 0.35
    Cluster 6: Size:750 | Avg:0.12 | Min:-0.03 | Max: 0.29
    Cluster 1: Size:625 | Avg:0.12 | Min:-0.04 | Max: 0.32
    Cluster 19: Size:376 | Avg:0.12 | Min:-0.08 | Max: 0.33
    Cluster 4: Size:625 | Avg:0.11 | Min:-0.07 | Max: 0.31
    Cluster 15: Size:435 | Avg:0.10 | Min:-0.11 | Max: 0.30
    Cluster 11: Size:756 | Avg:0.10 | Min:-0.06 | Max: 0.30
    Cluster

In [17]:
print("Самые популярные темы в кластере:")
for i in range(22):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=5)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")

Top terms per cluster (based on centroids):
Cluster 0: долг выделил платеж обязательного кредитам 
Cluster 1: российскому ким одновременно джеймса туркмении 
Cluster 2: армией сотрудником ахмед северный главнокомандующий 
Cluster 3: токио совершал акватории охранник авиакомпании 
Cluster 4: сочи вернулся некий star сына 
Cluster 5: малайзию следственный православной сосед информационный 
Cluster 6: александром кузнецов павел ухода ушел 
Cluster 7: финансовый рабочим расчетам нпз вместо 
Cluster 8: силами косово сбит басаева абхазии 
Cluster 9: фишер ельциным возглавит договорился роберт 
Cluster 10: взорвалась скончался приземлился мощный доставлен 
Cluster 11: прибытие находящийся временем аль китайского 
Cluster 12: честь занимался открыть форум провал 
Cluster 13: уильяма украинский альфа киев акб 
Cluster 14: вызван недавно турецкого павла приморья 
Cluster 15: абхазии захвачен уходили грозненского силами 
Cluster 16: борисом возглавил эдуард михаила приглашению 
Cluster 17: штат с

In [18]:
test_cluster = 10
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:10]:
    print(docs[d])
    print("-------------")

В Москве произошла крупная автокатастрофа | Все | В четверг около 21:10 в Северо-Западном административном округе Москвы произошла крупная автокатастрофа, сообщает РИА "Новости". На улице Народного ополчения в районе дома 50 столкнулись семь легковых автомобилей. Пострадали шесть человек, один из них скончался на месте, пятеро госпитализированы. Двоих пострадавших, зажатых в автомобилях, пришлось извлекать спасателям. Движение по улице Народного ополчения частично перекрыто, сотрудники милиции выясняют обстоятельства ДТП.
-------------
В центре Москвы столкнулись пять автомобилей | Все | В результате столкновения пяти автомобилей, произошедшего в воскресенье в Москве на Садовом кольце, погиб один человек, еще шестеро, в том числе двое детей, получили ранения различной степени тяжести. Одному из пострадавших была оказана медицинская помощь на месте, пятеро  госпитализированы. Об этом РИА "Новости" сообщили в ГИБДД Центрального округа Москвы. В другом дорожно-транспортном происшествии, с